# SQLCodeT5-ColNameAware model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q datasets rouge_score transformers==4.28.0

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

T5_MODEL = 'Salesforce/codet5-small'

tokenizer = AutoTokenizer.from_pretrained(T5_MODEL)
model = T5ForConditionalGeneration.from_pretrained(T5_MODEL)

In [ ]:
from datasets import load_dataset

train_data = load_dataset('wikisql', split='train+validation')
test_data = load_dataset('wikisql', split='test')

In [ ]:
def get_table_from_row(row):
  header = row['table']['header']
  data_types = row['table']['types']

  table_str = "Table(" + ", ".join([f"\'{h}\'" for h in header]) + ")"
  return table_str

def format_dataset(example):
  return {'input': 'translate to SQL the following natural language query: \'{}\', where the table is \'{}\''.format(example['question'], get_table_from_row(example)), 'target': example['sql']['human_readable']}

In [ ]:
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)

In [ ]:
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

In [ ]:
# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], pad_to_max_length=True, max_length=64)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], pad_to_max_length=True, max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask'],
    }

    return encodings

In [ ]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

In [ ]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import get_linear_schedule_with_warmup

PATH_TO_TRAINED_MODEL = '/content/drive/MyDrive/model0306_1'

In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=PATH_TO_TRAINED_MODEL,
    per_device_train_batch_size=128,
    num_train_epochs=15,
    per_device_eval_batch_size=128,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True
)

In [ ]:
from datasets import load_metric
rouge = load_metric("rouge")

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

  return {
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
  }

<ipython-input-13-ff41f60595c8>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
)

NameError: ignored

In [ ]:
trainer.evaluate()

{'eval_loss': 2.5842812061309814,
 'eval_rouge2_precision': 0.0068,
 'eval_rouge2_recall': 0.0016,
 'eval_rouge2_fmeasure': 0.0025,
 'eval_runtime': 95.0103,
 'eval_samples_per_second': 167.119,
 'eval_steps_per_second': 1.316}

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1,0.176700,0.086866,0.876500,0.817900,0.841200
2,0.096800,0.071974,0.889600,0.830700,0.854400
3,0.080300,0.065402,0.895600,0.839000,0.861700
4,0.070300,0.061048,0.899800,0.843100,0.865800
5,0.062900,0.058336,0.904200,0.845900,0.869400
6,0.057500,0.056869,0.907500,0.847700,0.871900
7,0.053400,0.054682,0.908200,0.849300,0.873100
8,0.050200,0.054154,0.909200,0.850700,0.874400
9,0.047000,0.053312,0.909700,0.851300,0.874800
10,0.044800,0.052743,0.910800,0.852100,0.875800


TrainOutput(global_step=7605, training_loss=0.06255210240681966, metrics={'train_runtime': 9786.951, 'train_samples_per_second': 99.279, 'train_steps_per_second': 0.777, 'total_flos': 1.643793849778176e+16, 'train_loss': 0.06255210240681966, 'epoch': 15.0})

In [ ]:
trainer.save_model(PATH_TO_TRAINED_MODEL)

In [ ]:
tokenizer.save_pretrained(PATH_TO_TRAINED_MODEL)

('/content/drive/MyDrive/model0306_1/tokenizer_config.json',
 '/content/drive/MyDrive/model0306_1/special_tokens_map.json',
 '/content/drive/MyDrive/model0306_1/vocab.json',
 '/content/drive/MyDrive/model0306_1/merges.txt',
 '/content/drive/MyDrive/model0306_1/added_tokens.json',
 '/content/drive/MyDrive/model0306_1/tokenizer.json')